In [1]:
# this notebook should stay in root
import os
import torch
import numpy as np
import pickle
import time
import random
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from generation_manager import GeneratorManager

In [2]:
curr_model_path = 'ckpt/model_TEST_flip.pth' # model checkpoint
# Saving only state_dict:
#torch.save(model.state_dict(), model_state_dict_path)
#model_state_dict_path = load_model_path + 'model_only_state_dict_v0Last.pth'
#print('Model state dict saved')

In [4]:
def save_sequences(predicted, predicted_stopped, tokens_probs_all, name):
    data_dir = "data_halogenase/generation_flip_model/"
    query = name
    os.makedirs(data_dir, exist_ok=True)
    
    predicted_data_file = os.path.join(data_dir, "predicted_data_" + query + ".p")
    with open(predicted_data_file, "wb") as file:
        pickle.dump(predicted, file)

    predicted_stopped_data_file = os.path.join(data_dir, "predicted_stopped_data_" + query + ".p")
    with open(predicted_stopped_data_file, "wb") as file:
        pickle.dump(predicted_stopped, file)

    tokens_probs_all_file = os.path.join(data_dir, "tokens_prob_data_" + query + ".p")
    with open(tokens_probs_all_file, "wb") as file:
        pickle.dump(tokens_probs_all, file)
        

In [5]:
# GENERATION parameters
temperature = 0.9
penalty = 1.2
top_p = 0.5
np.random.seed(1337)
torch.manual_seed(1337)
random.seed(1337)
torch.cuda.manual_seed_all(1337)

generator = GeneratorManager(curr_model_path, penalty=penalty, top_p=top_p, temperature=temperature)

MODEL SIZE: 
1280
Found PyTorch checkpoint at  ckpt/model_TEST_flip.pth
GPU aviable. Previous checkpoint loaded in GPU


In [6]:

def generation(name, keys):
    predicted = []
    predicted_stopped = []
    # true_tokens_index_in_probs_all = []
    tokens_probs_all = []
    # true_value = []
    seq_number = 0
    #for entry in random_selection:
    while seq_number <= 1000:
        # input_seq = entry['sequence']['metadata']['sequence']
        # FOR TESTIG the code:
        # input_seq = input_seq[0:24]
        input_seq = ''
        
        # if no tax keys:
        tax_lineage = keys
        offset = 0
        
        #print('generating sequences...')
        #start_time = time.time()
        res, tokens_generated_stopped, tokens_prob = generator.generation_complete_sequence_new(input_seq, tax_lineage, offset)
        #end_time = time.time()
        #elapsed_time = end_time - start_time
        #print(f"Done. Time taken: {elapsed_time} seconds.")
    
        tokens_probs_all.append(tokens_prob)
        
        # print("Input: ", input_seq)
        # print("Res: ", res)
        # true_value.append(input_seq)
        # true_tokens_index_in_prob = []
        # true_tokens_index_in_probs_all.append(true_tokens_index_in_prob)
        # tokens_probs_all.append(tokens_prob)
        # print('input_offset should be nothing: ', input_seq[:offset])
        predicted.append(input_seq[:offset] + res)
        if tokens_generated_stopped:
            for tmp_seq in tokens_generated_stopped:
                predicted_stopped.append(input_seq[:offset] + tmp_seq)
    
        seq_number += 1
        print('hello, this is seq_number', seq_number)
        print('hello, this is seq_number module: ', seq_number%10)
        
        # FOR TESTIG:
    
        
        if (seq_number%20) == 0:
            print('the model has generated: ', seq_number , ' sequences')
            print('saving sequences')
            save_sequences(predicted, predicted_stopped, tokens_probs_all, name)
    
            
    print('GENERATION ENDED')


In [ ]:
generation('p0_5_percent_penalty_0_key_only_hydro', [0])

In [ ]:
generation('p0_5_percent_penalty_0_key_0_1', [0, 1])

In [ ]:
generation('p0_5_percent_penalty_0_key_0_2', [0, 2])

In [ ]:
generation('p0_5_percent_penalty_0_key_0_3', [0, 3])